This notebook demonstrates the utility of `view_cell_labels()` for generating clear visualizations of various genes, observations, and their combinations. Additionally, it illustrates how to export these visualizations as images for easy sharing and display.

In [1]:
import warnings
warnings.simplefilter("ignore", FutureWarning)
import scanpy as sc
import bin2cell as b2c
from PIL import Image
import matplotlib
from matplotlib import rcParams
print(b2c.__version__) # make sure the b2c version is >=0.3.3 

0.3.3


# load b2c object 

In [2]:
# load annotated object
import os
path = '.../square_002um/'
os.chdir(path)
cdata = sc.read_h5ad('b2c.h5ad')

In [3]:
cdata

AnnData object with n_obs × n_vars = 257215 × 18058
    obs: 'bin_count', 'array_row', 'array_col', 'labels_joint_source'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
    uns: 'spatial'
    obsm: 'spatial', 'spatial_cropped'

# Predict cell types 

In [4]:
# norm and prefilter
import numpy as np
cdata.X.data = np.round(cdata.X.data) # correct back to counts
sc.pp.filter_genes(cdata, min_cells=3)
sc.pp.filter_cells(cdata, min_genes=50)
sc.pp.calculate_qc_metrics(cdata,inplace=True)
sc.pp.normalize_total(cdata,target_sum=1e4)
sc.pp.log1p(cdata)

In [5]:
# celltypist part
import celltypist
predictions = celltypist.annotate(cdata, model = 'Human_Colorectal_Cancer.pkl', majority_voting = False)
cdata = predictions.to_adata()

🔬 Input data has 250530 cells and 18057 genes
🔗 Matching reference genes in the model
🧬 3733 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!


# Plotting 

In [6]:
def save_outputs(image, legends, 
                 image_path="annotated_image.jpg", 
                 legend_paths=("fill_legend.jpg", "border_legend.jpg"), 
                 image_quality=75, 
                 image_compression=True):
    """
    Save the annotated image and legends to disk with optional compression.

    Parameters
    ----------
    image : numpy.ndarray
        The annotated image as a NumPy array.
    legends : dictionary of matplotlib.figure.Figure
        A list of legends as Matplotlib figures.
    image_path : str, optional
        Path to save the annotated image, by default "annotated_image.jpg".
    legend_paths : tuple of str, optional
        Paths to save the legends, by default ("fill_legend.jpg", "border_legend.jpg").
    image_quality : int, optional
        Quality of the saved image (1-100), by default 75.
    image_compression : bool, optional
        Whether to enable compression for the image, by default True.
    """

    import logging 
    logging.getLogger().setLevel(logging.WARNING)

    # Save the annotated image
    img_pil = Image.fromarray(image)
    img_pil.save(
        image_path,
        format="JPEG",
        quality=image_quality,
        optimize=image_compression
    )
    print(f"Image saved to {image_path} with quality={image_quality}, compression={image_compression}")

    # Save each legend
    for legend, legend_path in zip(legends.values(), legend_paths):
        # Ensure the file extension is PNG for better Matplotlib support
        if not legend_path.endswith(".pdf"):
            legend_path = legend_path.rsplit('.', 1)[0] + ".pdf"
        
        legend.savefig(legend_path, dpi=300, bbox_inches="tight")
        print(f"Legend saved to {legend_path}")

In [7]:
image_path = path+"/stardist/he.tiff"
labels_npz_path = path+"/stardist/he.npz"
matplotlib.rcParams['pdf.fonttype'] = 42
sc.settings.set_figure_params(dpi = 150, color_map = 'RdPu', dpi_save = 150, vector_friendly = True, format = 'pdf')

In [8]:
import time
t1=time.time()
# make annotated image of cell labels and confidance scores
fill_col='predicted_labels'
border_col='conf_score'
annotated_image, legends = b2c.view_cell_labels(image_path, labels_npz_path, cdata, fill_key=fill_col, border_key=border_col,cat_cmap='tab20') 
t2=time.time()
t2-t1

51.45907211303711

In [9]:
# Save outputs
save_outputs(
    image=annotated_image,
    legends=legends,
    image_path="annotated_image.jpg",
    legend_paths=("fill_legend.pdf", "border_legend.pdf"),
    image_quality=90,
    image_compression=True
)

Image saved to annotated_image.jpg with quality=90, compression=True
Legend saved to fill_legend.pdf
Legend saved to border_legend.pdf


# More plotting examples

In [10]:
# Plot CD19 gene espression from one cell type abd show only border with custom "jet" cmap
cells = ['CD19+CD20+ B']
ddata = cdata[cdata.obs['predicted_labels'].isin(cells)] # subset to these 3 cell types 
border_col = 'CD19' # gene that is activated in CRC cells
annotated_image, legends = b2c.view_cell_labels(image_path, labels_npz_path, ddata,border_key=border_col,cont_cmap='viridis') 
save_outputs(image=annotated_image, legends=legends, image_path=f"anno_img_{cells[0]+'_'+border_col}.jpg",legend_paths=(f"border_{border_col}",), image_quality=90)

Image saved to anno_img_CD19+CD20+ B_CD19.jpg with quality=90, compression=True
Legend saved to border_CD19.pdf


In [11]:
# Plot conf score from one cell type as fill and jet colormap 
# subset to cells of interest 
cells = ['CMS2']
ddata = cdata[cdata.obs['predicted_labels'].isin(cells)] # subset to this cell type 
fill_col = 'EPCAM' # gene that is activated in CRC cells
border_col = 'predicted_labels' # gene that is activated in CRC cells
annotated_image, legends = b2c.view_cell_labels(image_path, labels_npz_path, ddata,fill_key=fill_col, border_key=border_col,  cont_cmap='jet') 
save_outputs(image=annotated_image, legends=legends, image_path=f"anno_img_{cells[0]+'_'+fill_col}.jpg",legend_paths=(f"fill_{fill_col}",f"border_{border_col}"), image_quality=90)

Image saved to anno_img_CMS2_EPCAM.jpg with quality=90, compression=True
Legend saved to fill_EPCAM.pdf
Legend saved to border_predicted_labels.pdf
